** This is a work in progress **
## Objective:
1.	Analysing the Titanic dataset to obtain useful and interesting insights.
2.	Predicting survival using multiple ML techniques
After poking around for inspiration from fellow kagglers, i noticed Random Forest Trees to be the go-to technique for prediction in this competition.. Having python at my disposal, i figured i am interested in attempting multiple ML techniques and hope to analyze how they fare.

## Step 1:  Import Libraries

In [ ]:
# Data Analysis and Wrangling
import numpy as np 
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# allowing multiple scrollable outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Step 2: Load Data
we create a combined dataframe - Full, to allow us perform operations both datasets require, without repetition.

In [ ]:
# Load data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

# To perform manipulation that both datasets need in common
full = train.append(test,ignore_index = True) 


## Step 3: Preliminary/Descriptive Analysis 
Now that we have our data ready, we can look at the variables, shape of the data, missing values and some standard descriptive stats. Further, we can figure out sensible data cleaning and possible feature extraction/engineering opportunities.


In [ ]:
# preliminary analysis
print('Train:', train.shape, 'Test:', test.shape, "Full:", full.shape)
full.head()
full.isnull().sum()
full.describe()

* Train has 891 records, Test has 418 records (total: 1309), we will use Test to predict survival after building model(s).
* Missing values Age - 263 (~20%), Cabin - 1014 (~77%) , Embarked- 2, Fare 1. The missing values might be important in deciding the survival of a passenger, hence, the missingness has to be dealth with.
* Names have titles (Royalty, Mr, Master, Mrs and so on.) which might have some correlation with survival rates.

### Some initial data cleaning for better visualization to aid analysis

In [ ]:
#filling unknown cabin with U
full.Cabin = train.Cabin.fillna('U')

#convert male,female to 0,1
full['Sex'].replace(["male","female"],[0,1],inplace= True)

In [ ]:
#deciding importance of variables
corr = train.corr()
sns.heatmap(corr, annot= True, cmap="YlGnBu")
